In [2]:
import os
import sys
import importlib
import json
import glob
import numpy as np
import tensorflow as tf

import util_recognition_network
import util_cochlear_model

ARCH = 1
TASK = 'A'

fn_ckpt = 'models/recognition_networks/arch{}_task{}.ckpt*'.format(ARCH, TASK)
fn_ckpt = glob.glob(fn_ckpt)[-1].replace('.index', '')
fn_arch = 'models/recognition_networks/arch{}.json'.format(ARCH)
with open(fn_arch, 'r') as f_arch:
    list_layer_dict = json.load(f_arch)

fn_ckpt, fn_arch


('models/recognition_networks/arch1_taskA.ckpt-550000',
 'models/recognition_networks/arch1.json')

In [ ]:
importlib.reload(util_recognition_network)

tf.reset_default_graph()

if 'taskA' in fn_ckpt:
    n_classes_dict = {"/stimuli/labels_binary_via_int": 517}
else:
    n_classes_dict = {"/stimuli/word_int": 794}

var_scope = ''
with tf.variable_scope(var_scope):
    tensor_input = tf.placeholder(tf.float32, [None, 40, 20000, 1])
    tensor_output, tensors = util_recognition_network.build_network(
        tensor_input,
        list_layer_dict,
        n_classes_dict=n_classes_dict)

var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=var_scope)
var_dict = {v.name: v for v in var_list}

saver = tf.train.Saver(var_list=var_list, max_to_keep=0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    var_dict_init = sess.run(var_dict)
    saver.restore(sess, fn_ckpt)
    var_dict_load = sess.run(var_dict)
    
#     save_path = fn_ckpt.replace('trash/', '')
#     print('WRITING: {}'.format(save_path))
#     saver_to_save.save(
#         sess,
#         save_path=save_path,
#         global_step=None,
#         latest_filename=None,
#         meta_graph_suffix='meta',
#         write_meta_graph=False,
#         write_state=False,
#         strip_default_attrs=False)


In [ ]:
for v in var_list:
    print(v.name, np.mean(var_dict_init[v.name]), np.mean(var_dict_load[v.name]))


In [5]:
import util_cochlear_model
importlib.reload(util_cochlear_model)
tf.reset_default_graph()

tensor_waveform = tf.placeholder(tf.float32, [None, 40000])

util_cochlear_model.build_cochlear_model(
    tensor_waveform,
    signal_rate=20000,
    filter_type='half-cosine',
    filter_spacing='erb',
    HIGH_LIM=8000,
    LOW_LIM=20,
    N=40,
    SAMPLE_FACTOR=1,
    bandwidth_scale_factor=1.0,
    compression='stable_point3',
    include_highpass=False,
    include_lowpass=False,
    linear_max=1.0,
    rFFT=True,
    rectify_and_lowpass_subbands=True,
    return_subbands_only=True)


[make_cos_filters_nx] using filter_spacing=`erb`


(<tf.Tensor 'ExpandDims_1:0' shape=(?, 40, 20000, 1) dtype=float32>,
 {'fft_input': <tf.Tensor 'exd_fft_of_input:0' shape=(?, 1, 20001) dtype=complex64>,
  'filts_tensor': <tf.Tensor 'ExpandDims:0' shape=(1, 40, 20001) dtype=complex64>,
  'input_real': <tf.Tensor 'Placeholder:0' shape=(?, 40000) dtype=float32>,
  'input_signal': <tf.Tensor 'Placeholder:0' shape=(?, 40000) dtype=float32>,
  'output_tfcoch_graph': <tf.Tensor 'ExpandDims_1:0' shape=(?, 40, 20000, 1) dtype=float32>,
  'subbands': <tf.Tensor 'mul_subbands:0' shape=(?, 40, 20001) dtype=complex64>,
  'subbands_ifft': <tf.Tensor 'ifft_subbands:0' shape=(?, 40, 40000) dtype=float32>,
  'subbands_time': <tf.Tensor 'ifft_subbands:0' shape=(?, 40, 40000) dtype=float32>,
  'subbands_time_lowpassed': <tf.Tensor 'transpose_2:0' shape=(?, 40, 20000) dtype=float32>,
  'subbands_time_lowpassed_compressed': <tf.Tensor 'ExpandDims_1:0' shape=(?, 40, 20000, 1) dtype=float32>,
  'subbands_time_processed': <tf.Tensor 'ExpandDims_1:0' shape=(